## [Create EC2 Instance](https://blog.ipswitch.com/how-to-create-an-ec2-instance-with-python)


#### add [boto3 credentials](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html) to ~/.aws/credentials using a profile_name of form:

```
[default]
aws_access_key_id = XXXXXXXXXXXXXXXXXXXXXXXXXXXX
aws_secret_access_key = xxxxxxxxxxxxxxxxxxxxxxxxxxx

region=us-east-1


[seigniorage]
aws_access_key_id = XXXXXXXXXXXXXXXXXXXXXXXXXXX
aws_secret_access_key = xxxxxxxxxxxxxxxxxxxxxxxxx

region=us-east-2
```

In [ ]:
import boto3
import os


session = boto3.Session(profile_name='seigniorage')
ec2 = session.resource('ec2')

In [ ]:
KEY_NAME  =  "blockchain-nodes-keypair"
KEY_DIRECTORY = "/data/code/software/aws/seigniorage/test/"


SECURITYGROUP_NAME  =  "blockchain-nodes-sg"

DATAVOLUME_NAME     =  "Algorand Mainnet Chain Data"
DATAVOLUME_SIZE     =  64

INSTANCE_NAME = 'Algorand Mainnet'
Amazon_Ubuntu_AMI_18_04_LTS = 'ami-0d5d9d301c853a04a'
Instance_Type = 't2.medium' #micro=1GB, small=2GB, medium=4GB, large=8GB https://us-east-2.console.aws.amazon.com/ec2/v2/home?region=us-east-2#LaunchInstanceWizard:

AVZONE = session.region_name+'b'
VPC_ID, SUBNET_ID = get_subnet_id(AVZONE,"public")
SECURITY_GROUP_ID = get_security_group_id(session,VPC_ID,SECURITYGROUP_NAME)

In [ ]:
def get_subnet_id(AVZONE=session.region_name+'b',
                  subnet_type='public'):
    
    for vpc in ec2.vpcs.all():
        for az in ec2.meta.client.describe_availability_zones()["AvailabilityZones"]:
            for subnet in vpc.subnets.filter(Filters=[{"Name": "availabilityZone", "Values": [az["ZoneName"]]}]):
                #print(vpc.id, az["ZoneName"], subnet.id, subnet.tags[0]['Value'])
                if (az["ZoneName"] == AVZONE) & (subnet_type in subnet.tags[0]['Value']):
                    return vpc.id, subnet.id

def get_security_group_id(session,VPC_ID,SECURITYGROUP_NAME):
    client = boto3.client("ec2", region_name=session.region_name)
    return client.describe_security_groups(Filters = [{"Name":"vpc-id",
                                               "Values":[VPC_ID]
                                               },{
                                                "Name":"group-name",
                                                "Values":[SECURITYGROUP_NAME]
                                              }])\
                ['SecurityGroups'][0]['GroupId']

# Create EC2 Instance

1. Key Pair
2. Ubuntu Server

### Create Key Pair for EC2 Instance

In [ ]:
file = KEY_DIRECTORY + KEY_NAME + '.pem'
outfile = open(file,'w')

# call the boto ec2 function to create a key pair
key_pair = ec2.create_key_pair(KeyName=KEY_NAME)

# capture the key and store it in a file
KeyPairOut = str(key_pair.key_material)
print(KeyPairOut)
outfile.write(KeyPairOut)
outfile.close()

os.chmod(file,0o400)

### User data for ubuntu server initialization

In [ ]:
USERDATA = '''#!/bin/bash
sudo apt-get update

sudo mkfs -t xfs /dev/xvdf
sudo mkdir /data
sudo mount /dev/xvdf /data
sudo chown -R ubuntu:ubuntu /data

export CHAIN_DATA=/data/algorand.com
echo 'export CHAIN_DATA=/data/algorand.com'  >> /home/ubuntu/.bashrc

export PATH=$PATH:$CHAIN_DATA
echo 'export PATH=$PATH:/data/algorand.com'  >> /home/ubuntu/.bashrc

source ~/.bashrc

mkdir $CHAIN_DATA
sudo chown -R ubuntu:ubuntu /data

sudo ln -s $CHAIN_DATA /var/lib/algorand


sudo apt-get update
sudo apt-get install -y python3-pip

yes w | pip3 install py-algorand-sdk

sudo apt-get install -y golang-go
go get -u github.com/algorand/go-algorand-sdk/...

sudo apt-get install -y npm
npm install algosdk

'''

instances = ec2.create_instances(
    ImageId=Amazon_Ubuntu_AMI_18_04_LTS,
    MinCount=1,
    MaxCount=1,
    UserData=USERDATA,
    IamInstanceProfile={
        'Name': 'truepersons-webserver-role'
    },
    BlockDeviceMappings=[
        {
            'DeviceName': '/dev/xvdf',
            'Ebs': {
                'DeleteOnTermination': True,
                'VolumeSize': DATAVOLUME_SIZE,
                'VolumeType': 'gp2'
            },
        },
    ],
    InstanceType=Instance_Type,
    KeyName=KEY_NAME,
    Placement={'AvailabilityZone':AVZONE},
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': INSTANCE_NAME
                },
            ]
        },
    ],
    NetworkInterfaces=[{'SubnetId': SUBNET_ID, 
                     'DeviceIndex': 0, 
                     'AssociatePublicIpAddress': True, 
                     'Groups': [SECURITY_GROUP_ID]}])

### Install latest version of Algorand

In [ ]:
INSTALL = '''
sudo apt-get update
sudo apt-get install -y gnupg2 curl software-properties-common
curl -O https://releases.algorand.com/key.pub
sudo apt-key add key.pub
sudo add-apt-repository "deb https://releases.algorand.com/deb/ stable main"
sudo apt-get update
sudo apt-get install -y algorand
sudo mv $CHAIN_DATA/config.json.example $CHAIN_DATA/config.json
sudo chown -R algorand:algorand $CHAIN_DATA
sudo systemctl start algorand
algod -v
'''